In [4]:
from scipy.spatial import distance as dist
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import imutils
import dlib
import cv2
   
def eye_aspect_ratio(eye):
    # compute the euclidean distances between vertcil eye landmarks
    a = dist.euclidean(eye[1], eye[5])
    b = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal eye landmark
    c = dist.euclidean(eye[0], eye[3])
    return (a + b) / (2.0 * c)


thresh_value = 0.2 # threshold value of ear to detect drawsiness
frames_to_alarm_on = 10 #number of consecutive frames the eye must be below the threshold to turn on the alarm
counter = 0 #frame counter

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
alarm = "alarm.wav"

cap = cv2.VideoCapture("video.mp4")

while(True):
        ret, frame = cap.read()
        image = imutils.resize(frame, width=800)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect faces in the grayscale image
        rects = detector(gray, 1)
        # loop over the face detections
        for (i, rect) in enumerate(rects):
                # determine the facial landmarks for the face region, then
                # convert the facial landmark (x, y)-coordinates to a NumPy array
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                
                # extract the left and right eye coordinates
                rightEye = shape[36:42]
                leftEye = shape[42:48]
                             
                
                #calculate EAR
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0

                # compute the convex hull for the left and right eye
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)

                cv2.putText(image, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                       
                if ear < thresh_value:
                    counter += 1
                    if counter >= frames_to_alarm_on:
                        t = Thread(target=playsound.playsound,
                            args=(alarm,))
                        t.deamon = True
                        t.start()
                        cv2.putText(image, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    counter = 0
                    
        cv2.imshow('Drawsiness Detection Output',image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
